# GameTheory 19 - Jeux Combinatoires en Lean

**Kernel** : Lean 4 (WSL) - Notebooks Lean natifs

**Navigation** : [← GameTheory-18-Lean-NashExistence](GameTheory-18-Lean-NashExistence.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [GameTheory-20-Lean-SocialChoice →](GameTheory-20-Lean-SocialChoice.ipynb)

---

## Introduction

Ce notebook explore les **jeux combinatoires** formalises dans **mathlib4**. Contrairement aux jeux strategiques (Nash, strategies mixtes), les jeux combinatoires sont des jeux a information parfaite ou deux joueurs jouent alternativement jusqu'a ce qu'un joueur ne puisse plus jouer.

### Jeux strategiques vs Jeux combinatoires

| Aspect | Jeux strategiques | Jeux combinatoires |
|--------|-------------------|--------------------|
| **Joueurs** | N joueurs simultanement | 2 joueurs alternativement |
| **Information** | Peut etre incomplete | Parfaite |
| **Hasard** | Possible | Aucun |
| **Fin** | Gains continus | Un joueur gagne/perd |
| **Exemples** | Poker, Auctions | Echecs, Go, Nim |
| **Theorie** | Nash, mecanismes | Conway, Sprague-Grundy |

### Objectifs pedagogiques

1. Comprendre la structure inductive `PGame` de mathlib
2. Explorer le jeu de Nim et sa valeur de Grundy
3. Decouvrir le theoreme de Sprague-Grundy
4. Introduction aux nombres surreels

### Prerequis

- Notebooks 17-18 (definitions Lean, Brouwer)
- Notions de base en theorie des jeux
- Familiarite avec les types inductifs en Lean

### Duree estimee : 50 minutes

---

## Plan de ce Notebook

1. [Configuration](#1-configuration)
2. [Theorie des Jeux Combinatoires](#2-theorie)
3. [PGame dans Mathlib](#3-pgame)
4. [Nim et Sprague-Grundy](#4-nim)
5. [Nombres Surreels](#5-surreals)
6. [Exercices](#6-exercices)
7. [Solutions](#7-solutions)
8. [Resume](#8-resume)

<a id="1-configuration"></a>
## 1. Configuration

In [1]:
import sys
import os

lean_path = os.path.abspath('../SymbolicAI/Lean')
if lean_path not in sys.path:
    sys.path.insert(0, lean_path)

from lean_runner import LeanRunner

runner = LeanRunner(backend='auto', timeout=60)

def run_lean(code: str, show_code: bool = True):
    """Execute du code Lean et affiche le resultat."""
    result = runner.run(code)
    if show_code:
        print("--- Code Lean ---")
        for i, line in enumerate(code.strip().split('\n'), 1):
            print(f"{i:3d} | {line}")
        print("-" * 40)
    if result.output:
        print("Sortie:", result.output)
    if result.errors:
        print("Erreurs:" if not result.success else "Avertissements:", result.errors)
    print(f"[{'OK' if result.success else 'ECHEC'}]")
    return result

print(f"Backend: {runner.check_installation()['backend']}")
print("\nNote: Ce notebook reference du code mathlib4 qui necessite une installation complete.")
print("Les exemples simples fonctionnent sans mathlib.")

ModuleNotFoundError: No module named 'lean_runner'

<a id="2-theorie"></a>
## 2. Theorie des Jeux Combinatoires

### 2.1 Definition informelle

Un **jeu combinatoire** (au sens de Conway) est defini par :
- Deux joueurs : **Gauche (L)** et **Droite (R)**
- Un ensemble de **positions** legales
- Des **regles de deplacement** pour chaque joueur
- Une **condition de fin** : le joueur qui ne peut plus jouer perd

### 2.2 Classification

| Type | Description | Exemple |
|------|-------------|--------|
| **Impartial** | Memes coups pour les deux joueurs | Nim |
| **Partizan** | Coups differents selon le joueur | Echecs, Domineering |

### 2.3 Issue d'un jeu

Quatre issues possibles pour une position :

| Issue | Notation | Signification |
|-------|----------|---------------|
| **N** (Next) | $G > 0$ | Le premier joueur gagne |
| **P** (Previous) | $G < 0$ | Le second joueur gagne |
| **L** (Left) | $G \| 0$ | Gauche gagne (qui que ce soit) |
| **R** (Right) | $G \| 0$ | Droite gagne (incomparable) |
| **Zero** | $G = 0$ | Le joueur qui joue perd |

<a id="3-pgame"></a>
## 3. PGame dans Mathlib

### 3.1 Definition inductive

Dans mathlib4, un **PGame** (Pre-Game) est defini inductivement :

In [ ]:
run_lean("""
-- Definition simplifiee de PGame (la version mathlib est plus elaboree)

inductive SimplePGame where
  | mk : (L : Type) → (R : Type) → (L → SimplePGame) → (R → SimplePGame) → SimplePGame

-- L = type des coups de Gauche
-- R = type des coups de Droite
-- moveL : L → SimplePGame = fonction donnant la position apres un coup de Gauche
-- moveR : R → SimplePGame = fonction donnant la position apres un coup de Droite

#check SimplePGame

-- Le jeu "zero" : aucun coup pour personne
def zero : SimplePGame := SimplePGame.mk Empty Empty 
  (fun e => e.elim)   -- Pas de coup pour L
  (fun e => e.elim)   -- Pas de coup pour R

#check zero
""")

### 3.2 API mathlib

Voici les principales definitions de `Mathlib.SetTheory.PGame.Basic` :

In [ ]:
mathlib_pgame = """
-- Extrait de Mathlib.SetTheory.PGame.Basic
-- https://leanprover-community.github.io/mathlib4_docs/Mathlib/SetTheory/PGame/Basic.html

/-
-- Definition officielle dans mathlib4 :
inductive PGame
  | mk : (α β : Type u) → (α → PGame) → (β → PGame) → PGame

-- Accesseurs
def LeftMoves (G : PGame) : Type := ...
def RightMoves (G : PGame) : Type := ...
def moveLeft (G : PGame) : G.LeftMoves → PGame := ...
def moveRight (G : PGame) : G.RightMoves → PGame := ...

-- Jeux primitifs
instance : Zero PGame := ⟨PGame.mk PEmpty PEmpty PEmpty.elim PEmpty.elim⟩
instance : One PGame := ⟨PGame.mk PUnit PEmpty (fun _ => 0) PEmpty.elim⟩

-- Relations
def le (x y : PGame) : Prop := ...  -- x ≤ y
def lt (x y : PGame) : Prop := ...  -- x < y
def equiv (x y : PGame) : Prop := x ≤ y ∧ y ≤ x  -- x ≈ y

-- Operations
def neg : PGame → PGame  -- -G (echanger L et R)
def add : PGame → PGame → PGame  -- G + H (somme disjointe)
-/
"""

print("=" * 60)
print("API PGame dans Mathlib4")
print("=" * 60)
print(mathlib_pgame)

### 3.3 Exemples de jeux

Construisons quelques jeux simples :

In [ ]:
run_lean("""
-- Jeux simples (sans mathlib)

inductive PG where
  | mk : (L R : Type) → (L → PG) → (R → PG) → PG

-- 0 : aucun coup
def PG.zero : PG := PG.mk Empty Empty (fun e => e.elim) (fun e => e.elim)

-- 1 : Gauche peut bouger vers 0, Droite ne peut pas
def PG.one : PG := PG.mk Unit Empty (fun _ => PG.zero) (fun e => e.elim)

-- -1 : Droite peut bouger vers 0, Gauche ne peut pas
def PG.negOne : PG := PG.mk Empty Unit (fun e => e.elim) (fun _ => PG.zero)

-- * (star) : les deux peuvent bouger vers 0
def PG.star : PG := PG.mk Unit Unit (fun _ => PG.zero) (fun _ => PG.zero)

#check PG.zero
#check PG.one
#check PG.negOne
#check PG.star
""")

### 3.4 Interpretation des jeux primitifs

| Jeu | Notation | Qui gagne ? |
|-----|----------|-------------|
| $0$ | `{|}` | Second joueur (P-position) |
| $1$ | `{0|}` | Gauche (meme en second) |
| $-1$ | `{|0}` | Droite (meme en second) |
| $*$ | `{0|0}` | Premier joueur (N-position) |

<a id="4-nim"></a>
## 4. Nim et Sprague-Grundy

### 4.1 Le jeu de Nim

**Nim** est un jeu impartial classique :
- Plusieurs tas d'objets
- A chaque tour, un joueur retire au moins un objet d'un tas
- Le joueur qui ne peut plus jouer perd

### 4.2 Nim a un seul tas

In [ ]:
run_lean("""
-- Nim avec un tas de n objets
-- nim(0) = {|} = 0 (pas de coup)
-- nim(n) = {nim(0), nim(1), ..., nim(n-1) | nim(0), nim(1), ..., nim(n-1)}

inductive PG where
  | mk : (L R : Type) → (L → PG) → (R → PG) → PG

-- nim(n) : jeu impartial avec n objets
def nim : Nat → PG
  | 0 => PG.mk Empty Empty (fun e => e.elim) (fun e => e.elim)
  | n + 1 => PG.mk (Fin (n + 1)) (Fin (n + 1))
      (fun k => nim k.val)   -- Gauche peut aller vers nim(0), nim(1), ..., nim(n)
      (fun k => nim k.val)   -- Droite aussi (jeu impartial)

#check nim
#check nim 0  -- = 0
#check nim 1  -- = *
#check nim 3
""")

### 4.3 Valeur de Grundy

La **valeur de Grundy** (ou nimber) d'un jeu impartial $G$ est le plus petit entier $n$ tel que $G$ n'a pas de coup vers un jeu equivalent a $\text{nim}(n)$.

$$\text{grundy}(G) = \text{mex}\{\text{grundy}(G') : G' \text{ est un coup depuis } G\}$$

ou $\text{mex}$ (minimum excludant) est le plus petit entier non present dans l'ensemble.

In [ ]:
# Implementation Python de la valeur de Grundy

def mex(s: set) -> int:
    """Minimum excludant : plus petit entier >= 0 absent de s."""
    n = 0
    while n in s:
        n += 1
    return n

def grundy_nim(n: int) -> int:
    """Valeur de Grundy de nim(n)."""
    # nim(n) peut aller vers nim(0), nim(1), ..., nim(n-1)
    # grundy(nim(k)) = k (par recurrence)
    return mex({grundy_nim(k) for k in range(n)})

# Verification
print("Valeurs de Grundy pour nim(n):")
for n in range(8):
    print(f"  grundy(nim({n})) = {grundy_nim(n)}")

print("\n=> grundy(nim(n)) = n (comme attendu)")

### 4.4 Theoreme de Sprague-Grundy

**Theoreme (Sprague 1935, Grundy 1939)** : Tout jeu impartial est equivalent a un jeu de Nim.

$$G \equiv \text{nim}(\text{grundy}(G))$$

**Corollaire** : Pour une somme de jeux impartiaux :
$$\text{grundy}(G_1 + G_2 + ... + G_n) = \text{grundy}(G_1) \oplus \text{grundy}(G_2) \oplus ... \oplus \text{grundy}(G_n)$$

ou $\oplus$ est le XOR binaire.

In [ ]:
# Application : Nim a plusieurs tas

def nim_sum(*heaps):
    """Valeur de Grundy d'un jeu de Nim a plusieurs tas."""
    result = 0
    for h in heaps:
        result ^= h  # XOR
    return result

def nim_winner(*heaps):
    """Determine qui gagne au Nim avec les tas donnes."""
    g = nim_sum(*heaps)
    if g == 0:
        return "Second joueur (P-position)"
    else:
        return f"Premier joueur (N-position, Grundy = {g})"

# Exemples
print("Nim avec tas [3, 5, 7]:")
print(f"  Grundy = {nim_sum(3, 5, 7)} = {bin(3)} XOR {bin(5)} XOR {bin(7)}")
print(f"  Gagnant: {nim_winner(3, 5, 7)}")

print("\nNim avec tas [1, 2, 3]:")
print(f"  Grundy = {nim_sum(1, 2, 3)} = 1 XOR 2 XOR 3")
print(f"  Gagnant: {nim_winner(1, 2, 3)}")

### 4.5 Formalisation dans mathlib

In [ ]:
mathlib_nim = """
-- Extrait de Mathlib.SetTheory.Game.Nim
-- https://leanprover-community.github.io/mathlib4_docs/Mathlib/SetTheory/Game/Nim.html

/-
-- Definition de nim dans mathlib4
def nim (n : Ordinal) : PGame :=
  PGame.mk (Set.Iio n) (Set.Iio n)
    (fun i => nim i.val)
    (fun i => nim i.val)

-- Valeur de Grundy
def grundyValue (G : Game) : Ordinal := ...

-- Theoreme de Sprague-Grundy
theorem equiv_nim_grundyValue (G : Game) [G.Impartial] :
    G ≈ nim (grundyValue G) := ...

-- Addition de nimbers = XOR
theorem nim_add_nim (n m : Ordinal) :
    nim n + nim m ≈ nim (n ^^^ m) := ...
-/
"""

print("=" * 60)
print("Nim et Sprague-Grundy dans Mathlib4")
print("=" * 60)
print(mathlib_nim)

<a id="5-surreals"></a>
## 5. Nombres Surreels

### 5.1 De Conway a Surreal

Les **nombres surreels** sont une extension des nombres reels decouverte par Conway en etudiant les jeux combinatoires.

Un nombre surreal est un jeu ou les deux joueurs ont les memes options (pas de choix asymetrique).

### 5.2 Construction

$$x = \{X_L | X_R\}$$

ou $X_L$ et $X_R$ sont des ensembles de surreels, et tout element de $X_L$ est strictement inferieur a tout element de $X_R$.

### 5.3 Exemples

| Jeu | Valeur | Construction |
|-----|--------|-------------|
| $\{|\}$ | $0$ | Aucune option |
| $\{0|\}$ | $1$ | Gauche peut aller a 0 |
| $\{|0\}$ | $-1$ | Droite peut aller a 0 |
| $\{0|1\}$ | $1/2$ | Entre 0 et 1 |
| $\{0,1|\}$ | $2$ | Gauche peut aller a 0 ou 1 |

In [ ]:
mathlib_surreal = """
-- Extrait de Mathlib.SetTheory.Surreal.Basic
-- https://leanprover-community.github.io/mathlib4_docs/Mathlib/SetTheory/Surreal/Basic.html

/-
-- Un PGame est numerique s'il satisfait certaines conditions
def Numeric (G : PGame) : Prop := ...

-- Le type Surreal est le quotient des jeux numeriques par l'equivalence
def Surreal := Quotient (setoid.mk (· ≈ ·) ...)

-- Embeddings
def Surreal.ofNat : ℕ → Surreal := ...
def Surreal.ofInt : ℤ → Surreal := ...

-- Operations
instance : Add Surreal := ...
instance : Neg Surreal := ...
instance : Mul Surreal := ...

-- Les surreels forment un corps ordonne
instance : LinearOrderedField Surreal := ...
-/
"""

print("=" * 60)
print("Nombres Surreels dans Mathlib4")
print("=" * 60)
print(mathlib_surreal)

<a id="6-exercices"></a>
## 6. Exercices

### Exercice 1 : Construire des jeux simples

En utilisant la structure `PG`, construire :
1. Le jeu $2 = \{1|\}$
2. Le jeu $1/2 = \{0|1\}$
3. Le jeu $\uparrow = \{0|*\}$ (up)

### Exercice 2 : Strategie optimale au Nim

Pour le Nim avec tas [5, 7, 11] :
1. Calculer la valeur de Grundy
2. Determiner le gagnant optimal
3. Trouver un coup gagnant pour le premier joueur

### Exercice 3 : Jeu de soustraction

Dans le **jeu de soustraction** $S(\{1, 3, 4\})$, on a un tas et on peut retirer 1, 3 ou 4 objets.
1. Calculer les valeurs de Grundy pour $n = 0, 1, ..., 12$
2. Identifier la periodicite

<a id="7-solutions"></a>
## 7. Solutions

### Solution Exercice 1

In [ ]:
run_lean("""
-- Solution Exercice 1 : Construire des jeux

inductive PG where
  | mk : (L R : Type) → (L → PG) → (R → PG) → PG

def PG.zero : PG := PG.mk Empty Empty (fun e => e.elim) (fun e => e.elim)
def PG.one : PG := PG.mk Unit Empty (fun _ => PG.zero) (fun e => e.elim)
def PG.star : PG := PG.mk Unit Unit (fun _ => PG.zero) (fun _ => PG.zero)

-- 1. Le jeu 2 = {1|}
def PG.two : PG := PG.mk Unit Empty (fun _ => PG.one) (fun e => e.elim)

-- 2. Le jeu 1/2 = {0|1}
def PG.half : PG := PG.mk Unit Unit (fun _ => PG.zero) (fun _ => PG.one)

-- 3. Le jeu up = {0|*}
def PG.up : PG := PG.mk Unit Unit (fun _ => PG.zero) (fun _ => PG.star)

#check PG.two
#check PG.half
#check PG.up
""")

### Solution Exercice 2

In [ ]:
# Solution Exercice 2 : Nim [5, 7, 11]

heaps = [5, 7, 11]
g = nim_sum(*heaps)

print(f"Tas: {heaps}")
print(f"\n1. Valeur de Grundy:")
print(f"   {heaps[0]} XOR {heaps[1]} XOR {heaps[2]}")
print(f"   = {bin(heaps[0])} XOR {bin(heaps[1])} XOR {bin(heaps[2])}")
print(f"   = {g} ({bin(g)})")

print(f"\n2. Gagnant: Premier joueur (car Grundy != 0)")

print(f"\n3. Coup gagnant:")
print(f"   On cherche un tas qu'on peut modifier pour rendre Grundy = 0")
print(f"   Pour chaque tas h, on veut h' tel que h' XOR (autres) = 0")
print(f"   Donc h' = XOR(autres)")

for i, h in enumerate(heaps):
    others = nim_sum(*[heaps[j] for j in range(len(heaps)) if j != i])
    target = others  # h' tel que h' XOR others = 0
    if target < h:  # On peut seulement reduire
        new_heaps = heaps.copy()
        new_heaps[i] = target
        print(f"   - Reduire tas {i} de {h} a {target}: {new_heaps}")
        print(f"     Verification: Grundy = {nim_sum(*new_heaps)}")

### Solution Exercice 3

In [ ]:
# Solution Exercice 3 : Jeu de soustraction S({1, 3, 4})

def grundy_subtraction(n: int, moves: set, memo: dict = None) -> int:
    """Valeur de Grundy du jeu de soustraction."""
    if memo is None:
        memo = {}
    if n in memo:
        return memo[n]
    if n == 0:
        return 0
    
    reachable = set()
    for m in moves:
        if n - m >= 0:
            reachable.add(grundy_subtraction(n - m, moves, memo))
    
    result = mex(reachable)
    memo[n] = result
    return result

moves = {1, 3, 4}
print(f"Jeu de soustraction S({moves})")
print(f"\n1. Valeurs de Grundy:")

grundy_values = []
for n in range(20):
    g = grundy_subtraction(n, moves)
    grundy_values.append(g)
    print(f"   G({n:2d}) = {g}")

print(f"\n2. Sequence: {grundy_values}")
print(f"\n   Pattern: La sequence semble periodique de periode 7 apres n=0")
print(f"   [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, ...] - periodicite partielle")

<a id="8-resume"></a>
## 8. Resume

### Concepts cles

| Concept | Definition | Exemple |
|---------|------------|--------|
| **PGame** | Jeu defini inductivement par coups L/R | $\{0|\} = 1$ |
| **Nim** | Jeu impartial avec tas | nim(3) = \{0,1,2 | 0,1,2\} |
| **Grundy** | mex des valeurs des successeurs | grundy(nim(n)) = n |
| **Sprague-Grundy** | Tout jeu impartial ~ nim(grundy) | |
| **Surreal** | Jeu numerique quotiente | 1/2 = \{0|1\} |

### API mathlib4

| Module | Contenu |
|--------|--------|
| `Mathlib.SetTheory.PGame.Basic` | Definition PGame, relations |
| `Mathlib.SetTheory.Game.Nim` | nim, grundyValue, Sprague-Grundy |
| `Mathlib.SetTheory.Surreal.Basic` | Nombres surreels |

### Prochaines etapes

Dans le notebook suivant (**GameTheory-20-Lean-SocialChoice**), nous explorerons :
- Le theoreme d'impossibilite d'Arrow
- Le theoreme de Sen
- Le theoreme de l'electeur median
- Port de lean-social-choice vers Lean 4

---

**Navigation** : [← GameTheory-18-Lean-NashExistence](GameTheory-18-Lean-NashExistence.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [GameTheory-20-Lean-SocialChoice →](GameTheory-20-Lean-SocialChoice.ipynb)